# Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Phan Thiet,VN,10.93,108.10,75.51,88,100,3.33,'overcast clouds'
1,1,Cape Town,ZA,-33.93,18.42,66.99,72,0,8.05,'clear sky'
2,2,Georgetown,MY,5.41,100.34,78.80,88,40,3.36,'scattered clouds'
3,3,Victoria,HK,22.29,114.16,75.00,71,90,7.49,'overcast clouds'
4,4,Hobart,AU,-42.88,147.33,64.00,72,75,9.17,'broken clouds'


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Phan Thiet,VN,10.93,108.10,75.51,88,100,3.33,'overcast clouds'
1,1,Cape Town,ZA,-33.93,18.42,66.99,72,0,8.05,'clear sky'
2,2,Georgetown,MY,5.41,100.34,78.80,88,40,3.36,'scattered clouds'
3,3,Victoria,HK,22.29,114.16,75.00,71,90,7.49,'overcast clouds'
5,5,Huichapan,MX,20.38,-99.65,71.60,28,5,26.40,'clear sky'
6,6,Puerto Gaitan,CO,4.31,-72.08,74.52,91,100,6.29,'overcast clouds'
7,7,Cap Malheureux,MU,-19.98,57.61,66.99,69,46,3.00,'scattered clouds'
10,10,Port Hedland,AU,-20.32,118.57,75.20,78,90,8.05,'overcast clouds'
11,11,Joao Pinheiro,BR,-17.74,-46.17,70.16,89,16,5.10,'light rain'
14,14,Arraial Do Cabo,BR,-22.97,-42.03,71.60,64,75,14.99,'broken clouds'


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                242
City                   242
Country                242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
     # Done in step #2 with the'dropna()'

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Phan Thiet,VN,75.51,'overcast clouds',10.93,108.10,
1,Cape Town,ZA,66.99,'clear sky',-33.93,18.42,
2,Georgetown,MY,78.80,'scattered clouds',5.41,100.34,
3,Victoria,HK,75.00,'overcast clouds',22.29,114.16,
5,Huichapan,MX,71.60,'clear sky',20.38,-99.65,
6,Puerto Gaitan,CO,74.52,'overcast clouds',4.31,-72.08,
7,Cap Malheureux,MU,66.99,'scattered clouds',-19.98,57.61,
10,Port Hedland,AU,75.20,'overcast clouds',-20.32,118.57,
11,Joao Pinheiro,BR,70.16,'light rain',-17.74,-46.17,
14,Arraial Do Cabo,BR,71.60,'broken clouds',-22.97,-42.03,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"       
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Phan Thiet,VN,75.51,'overcast clouds',10.93,108.10,Hotel Saigon-PT
1,Cape Town,ZA,66.99,'clear sky',-33.93,18.42,Southern Sun Waterfront Cape Town
2,Georgetown,MY,78.80,'scattered clouds',5.41,100.34,Cititel Penang
3,Victoria,HK,75.00,'overcast clouds',22.29,114.16,Mini Hotel Central
5,Huichapan,MX,71.60,'clear sky',20.38,-99.65,Hotel Boutique Casa Bixi
6,Puerto Gaitan,CO,74.52,'overcast clouds',4.31,-72.08,Brisas del Yucao
7,Cap Malheureux,MU,66.99,'scattered clouds',-19.98,57.61,Pereybere Hotel & Spa
10,Port Hedland,AU,75.20,'overcast clouds',-20.32,118.57,The Esplanade Hotel
11,Joao Pinheiro,BR,70.16,'light rain',-17.74,-46.17,Hotel Santos
14,Arraial Do Cabo,BR,71.60,'broken clouds',-22.97,-42.03,Pousada Porto Praia


In [12]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
current_weather = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=current_weather,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))